# Patent Resolver By Angel

In [1]:
from google_patent_scraper import scraper_class
import lxml
from pprint import pprint
import requests
import pandas as pd
import json

## Functions

In [67]:
def create_URL(search_criteria):
    return f'https://patents.google.com/xhr/parse?text={str(search_criteria)}&cursor=0&exp='
def find_appnum(data, id_):
    r = data.loc[data['doc_id'].isin([id_])]
    r = r['appnum']
    #print(r.values)
    try:
        return str(r.values[0])
    except:
        return None
def from_google_get_json(search_criteria):
    response = requests.get(create_URL(search_criteria))
    pprint(json_)
    json_ = response.json()
    if json_['error_no_patents_found']:
        return None
    else:
        results = json_['results']
        if len(result)>0:
            print(json_)
        result = results[0]
        return result
def from_uspto_get_json(search_criteria, oldpatentno):
    url = f'https://api.patentsview.org/patents/query?q={{"app_number":"{search_criteria}"}}&f=["patent_abstract","patent_title","patent_number"]'
    response = requests.get(url)
    if response.json()['patents'] is not None:
        d = response.json()['patents'][0]
        my_dict = {"id":d['patent_number'],'title':d['patent_title'],'abstract':d['patent_abstract']}
        return (True, my_dict)
    else:
        #print(f'error {patentno}')
        return (False, (oldpatentno,search_criteria))

## Load Data

In [3]:
data = pd.read_csv("patents.csv")

In [4]:
with open('unsuccessfull.json') as f:
    unsuccessful = json.load(f)

In [5]:
with open('successfull.json') as f:
    successfull = json.load(f)

In [6]:
seeds_ = unsuccessful['seeds']
antiseeds_ = unsuccessful['antiseeds']
antiseeds = successfull['antiseeds']
seeds = successfull['seeds']

In [33]:
errors = []
patents = []
for pno in seeds_:
    appnum = find_appnum(data, pno)
    success, patent = from_uspto_get_json(appnum, pno)
    if success:
        patents.append(patent)
    else:
        errors.append(patent)
print(len(errors))

164


In [34]:
final_patents = {'seeds':patents}
final_errors = {'seeds':errors}

In [35]:
errors = []
patents = []
for pno in antiseeds_:
    appnum = find_appnum(data, pno)
    success, patent = from_uspto_get_json(appnum, pno)
    if success:
        patents.append(patent)
    else:
        errors.append(patent)

In [36]:
final_patents['antiseeds'] = patents
final_errors['antiseeds'] = errors

In [71]:
pprint(len(final_patents['seeds'])+len(seeds))
pprint(len(final_patents['antiseeds'])+len(antiseeds))

736
780


In [72]:
# #pprint(final_errors['seeds'])
# # for entry in final_errors['seeds']:
# #     pprint(from_google_get_json(entry[1]))
# search_criteria = final_errors['seeds'][1][1]
# # print(search_criteria[1])
# print(create_URL(search_criteria))
# response = requests.get(create_URL(search_criteria))
# # print(2)
# json_ = response.json()
# pprint(json_)

In [81]:
final_seeds = final_patents['seeds'] + seeds
final_antiseeds = final_patents['antiseeds'] + antiseeds
print(len(final_seeds))
print(len(final_antiseeds))

736
780


In [82]:
successfull = {"seed": final_seeds, "antiseed":final_antiseeds}

In [83]:
import json
with open('successfull.json', 'w') as f:
    json.dump(successfull, f)

In [86]:
print(len(final_errors['antiseeds']))

120


In [87]:
import json
with open('unsuccessfull.json', 'w') as f:
    json.dump(final_errors, f)